In [31]:
import re, string, unicodedata
import nltk
import contractions # expanding contractions
import inflect # natural language related tasks of generating plurals, singular nouns, etc.
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from pymongo import MongoClient
import string


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Stagiaire2014\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Stagiaire2014\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Stagiaire2014\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Connect to database

In [32]:
try:
    client = MongoClient('localhost', 27017) # host, port
except ConnectionFailure as e:
    sys.stderr.write("Could not connect to MongoDB: %s" % e)
    sys.exit(1)

# create database tweet_database
db = client.tweets_database

# delete collection
#db.tweets.drop()

# create collection tweets (eq. of a table in relational database)
tweets = db.tweets

# number of tweets
print("total number of tweets:", tweets.count())
print("number of english tweets:", tweets.find({'lang': 'en'}).count())

total number of tweets: 3364001
number of english tweets: 2706977


### import path to use preprocess library (only for jupyter notebook)

In [33]:
import sys
path = 'D:\A_AHNE1\Tweet-Classification-Diabetes-Distress\preprocess'

if path not in sys.path:
    sys.path.insert(0, path)
sys.path

#sys.path.remove(path)
#sys.path

['D:\\A_AHNE1\\Tweet-Classification-Diabetes-Distress\\preprocess',
 '',
 'c:\\users\\stagiaire2014\\miniconda3\\envs\\deepscience\\python35.zip',
 'c:\\users\\stagiaire2014\\miniconda3\\envs\\deepscience\\DLLs',
 'c:\\users\\stagiaire2014\\miniconda3\\envs\\deepscience\\lib',
 'c:\\users\\stagiaire2014\\miniconda3\\envs\\deepscience',
 'c:\\users\\stagiaire2014\\miniconda3\\envs\\deepscience\\lib\\site-packages',
 'c:\\users\\stagiaire2014\\miniconda3\\envs\\deepscience\\lib\\site-packages\\xgboost-0.7-py3.5.egg',
 'c:\\users\\stagiaire2014\\miniconda3\\envs\\deepscience\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\Stagiaire2014\\.ipython']

### import class for the preprocessing part

In [34]:
from preprocess import Preprocess
prep = Preprocess()

### get first 1000 tweets

In [46]:
count = 1
tweets_list = ["I love:D python 😄 :-)"] # add sample tweet to illustrate emoticons and emojis 
for tweet in tweets.find({'lang': 'en'}):
    
    tweet = prep.get_text(tweet)
    tweets_list.append(tweet)
    count += 1

    if (count == 1000):
        break

### Preprocess tweets

In [47]:
for tweet in tweets_list:

    print("")
    print("")
    print("----------ORIGINAL----------:")
    print(tweet)
    print("")
    
    tweet = prep.replace_contractions(tweet)
    tweet = prep.replace_hashtags_URL_USER(tweet)
    tweet = prep.tokenize(tweet)
    tweet = prep.remove_punctuation(tweet)
    print("-----REPLACED CONTRACTIONS, HASHTAGS, URLs AND USER MENTIONS, TOKENISE; REMOVE PUNCTUATIONS----:")
    print(tweet)
    print("")
    
    tweet = prep.preprocess_emojis(tweet)
    tweet = prep.preprocess_emoticons(tweet)
    tweet = prep.remove_non_ascii(tweet)
    tweet = prep.to_lowercase(tweet)
    tweet = prep.replace_numbers(tweet)
    print("-----REPLACE EMOJIS AND EMOTICONS; REMOVE NON ASCII, TO LOWERCASE, REPLACE NUMBERS------:")
    print(tweet)
    print("")
    
    tweet = prep.remove_stopwords(tweet)
    tweet = prep.lemmatize_verbs(tweet)
    tweet = prep.stem_words(tweet)
    print("----------REMOVE STOPWORDS, LEMMATIZE AND STEM-------------:")
    print(tweet)
    print("")
    
    #tweets_list.append(tweet)
    #print(tweet)
    




----------ORIGINAL----------:
I love:D python 😄 :-)

-----REPLACED CONTRACTIONS, HASHTAGS, URLs AND USER MENTIONS, TOKENISE; REMOVE PUNCTUATIONS----:
['I', 'love', ':D', 'python', '😄', ':-)']

-----REPLACE EMOJIS AND EMOTICONS; REMOVE NON ASCII, TO LOWERCASE, REPLACE NUMBERS------:
['i', 'love', 'EMOT_LAUGH', 'python', 'EMOT_LAUGH', 'EMOT_SMILE']

----------REMOVE STOPWORDS, LEMMATIZE AND STEM-------------:
['love', 'EMOT_LAUGH', 'python', 'EMOT_LAUGH', 'EMOT_SMILE']



----------ORIGINAL----------:
NEW! Reduce Age-Related Diseases! Linked to reducing cancer, diabetes, and cardiovascular disease. MORE INFO @… https://t.co/z2zdz1uYsd

-----REPLACED CONTRACTIONS, HASHTAGS, URLs AND USER MENTIONS, TOKENISE; REMOVE PUNCTUATIONS----:
['NEW', 'Reduce', 'Age-Related', 'Diseases', 'Linked', 'to', 'reducing', 'cancer', 'diabetes', 'and', 'cardiovascular', 'disease', 'MORE', 'INFO', 'URL']

-----REPLACE EMOJIS AND EMOTICONS; REMOVE NON ASCII, TO LOWERCASE, REPLACE NUMBERS------:
['new', 'reduc

RT @dcarbohydrated: In pts with established CVD, we need more than just blood glucose control - Schenthaner #ADIT2017

-----REPLACED CONTRACTIONS, HASHTAGS, URLs AND USER MENTIONS, TOKENISE; REMOVE PUNCTUATIONS----:
['RT', 'USER', 'In', 'pts', 'with', 'established', 'CVD', 'we', 'need', 'more', 'than', 'just', 'blood', 'glucose', 'control', 'Schenthaner', 'ADIT', '2017']

-----REPLACE EMOJIS AND EMOTICONS; REMOVE NON ASCII, TO LOWERCASE, REPLACE NUMBERS------:
['rt', 'USER', 'in', 'pts', 'with', 'established', 'cvd', 'we', 'need', 'more', 'than', 'just', 'blood', 'glucose', 'control', 'schenthaner', 'adit', 'two thousand and seventeen']

----------REMOVE STOPWORDS, LEMMATIZE AND STEM-------------:
['rt', 'USER', 'pts', 'establish', 'cvd', 'need', 'blood', 'glucos', 'control', 'schenthan', 'adit', 'two thousand and seventeen']



----------ORIGINAL----------:
@perthdiabetes @NOdrugsdiabetes  @Type2Diabetes2 Unbeatable Type 2 Diabetic Cure By Naturally at Home : https://t.co/xtl6HFYc3e ,

-----REPLACED CONTRACTIONS, HASHTAGS, URLs AND USER MENTIONS, TOKENISE; REMOVE PUNCTUATIONS----:
['RT', 'USER', 'If', 'Americans', 'stopped', 'drinking', 'soda', 'every', 'day', '2', 'million', 'cases', 'of', 'diabetes', 'could', 'be', 'prevented', 'by', '2020']

-----REPLACE EMOJIS AND EMOTICONS; REMOVE NON ASCII, TO LOWERCASE, REPLACE NUMBERS------:
['rt', 'USER', 'if', 'americans', 'stopped', 'drinking', 'soda', 'every', 'day', 'two', 'million', 'cases', 'of', 'diabetes', 'could', 'be', 'prevented', 'by', 'two thousand and twenty']

----------REMOVE STOPWORDS, LEMMATIZE AND STEM-------------:
['rt', 'USER', 'american', 'stop', 'drink', 'soda', 'everi', 'day', 'two', 'million', 'case', 'diabet', 'could', 'prevent', 'two thousand and twenti']



----------ORIGINAL----------:
PLEASE SHARE: Have your say about the diabetic eye screening service in Bury with this short survey… https://t.co/RscjPqNk52

-----REPLACED CONTRACTIONS, HASHTAGS, URLs AND USER MENTIONS, TOKENISE; REMOVE PUNCTUAT

RT @simondankel: Important review of role of #insulin in #obesity. Time for a #paradigmshift! @whsource @davidludwigmd @FatEmperor https://…

-----REPLACED CONTRACTIONS, HASHTAGS, URLs AND USER MENTIONS, TOKENISE; REMOVE PUNCTUATIONS----:
['RT', 'USER', 'Important', 'review', 'of', 'role', 'of', 'insulin', 'in', 'obesity', 'Time', 'for', 'a', 'paradigmshift', 'USER', 'USER', 'USER', 'URL']

-----REPLACE EMOJIS AND EMOTICONS; REMOVE NON ASCII, TO LOWERCASE, REPLACE NUMBERS------:
['rt', 'USER', 'important', 'review', 'of', 'role', 'of', 'insulin', 'in', 'obesity', 'time', 'for', 'a', 'paradigmshift', 'USER', 'USER', 'USER', 'URL']

----------REMOVE STOPWORDS, LEMMATIZE AND STEM-------------:
['rt', 'USER', 'import', 'review', 'role', 'insulin', 'obes', 'time', 'paradigmshift', 'USER', 'USER', 'USER', 'URL']



----------ORIGINAL----------:
RT @MedscapePsych: Why sleep quality matters for patients with #diabetes: https://t.co/VZ9KAUbqsO

-----REPLACED CONTRACTIONS, HASHTAGS, URLs AND USE

-----REPLACED CONTRACTIONS, HASHTAGS, URLs AND USER MENTIONS, TOKENISE; REMOVE PUNCTUATIONS----:
['RT', 'USER', 'The', 'epidemiology', 'of', 'diabetes', 'in', 'pregnancy', 'ABCDBelfast', 'URL']

-----REPLACE EMOJIS AND EMOTICONS; REMOVE NON ASCII, TO LOWERCASE, REPLACE NUMBERS------:
['rt', 'USER', 'the', 'epidemiology', 'of', 'diabetes', 'in', 'pregnancy', 'abcdbelfast', 'URL']

----------REMOVE STOPWORDS, LEMMATIZE AND STEM-------------:
['rt', 'USER', 'epidemiolog', 'diabet', 'pregnanc', 'abcdbelfast', 'URL']



----------ORIGINAL----------:
RT @teamnovonordisk: Follow the world's first all-#diabetes pro cycling team as we race to change diabetes at @AmgenTOC!

#AMGENTOC 🇺🇸 🚴

-----REPLACED CONTRACTIONS, HASHTAGS, URLs AND USER MENTIONS, TOKENISE; REMOVE PUNCTUATIONS----:
['RT', 'USER', 'Follow', 'the', "world's", 'first', 'all-diabetes', 'pro', 'cycling', 'team', 'as', 'we', 'race', 'to', 'change', 'diabetes', 'at', 'USER', 'AMGENTOC', '🇺', '🇸', '🚴']

INFO: No category set for emoj

...

-----REPLACED CONTRACTIONS, HASHTAGS, URLs AND USER MENTIONS, TOKENISE; REMOVE PUNCTUATIONS----:
['RT', 'USER', 'First', 'speaker', 'in', 'the', 'eHealth', 'Adoption', 'Awards', 'is', 'USER', 'A', 'self-managment', 'diabetes', 'platform', 'from', 'NHS', 'Scotl']

-----REPLACE EMOJIS AND EMOTICONS; REMOVE NON ASCII, TO LOWERCASE, REPLACE NUMBERS------:
['rt', 'USER', 'first', 'speaker', 'in', 'the', 'ehealth', 'adoption', 'awards', 'is', 'USER', 'a', 'self-managment', 'diabetes', 'platform', 'from', 'nhs', 'scotl']

----------REMOVE STOPWORDS, LEMMATIZE AND STEM-------------:
['rt', 'USER', 'first', 'speaker', 'ehealth', 'adopt', 'award', 'USER', 'self-manag', 'diabet', 'platform', 'nhs', 'scotl']



----------ORIGINAL----------:
RT @teamnovonordisk: Great to meet Jackson- diagnosed with #diabetes 1 month ago &amp; show him diabetes doesn't have to stop his dreams!… 

-----REPLACED CONTRACTIONS, HASHTAGS, URLs AND USER MENTIONS, TOKENISE; REMOVE PUNCTUATIONS----:
['RT', 'USER', 'Great',

@maryportas Hi Mary! Im an indy designer from MCR about to launch my debut clothing line for diabetic girls... would you mentor me? X

-----REPLACED CONTRACTIONS, HASHTAGS, URLs AND USER MENTIONS, TOKENISE; REMOVE PUNCTUATIONS----:
['USER', 'Hi', 'Mary', 'I', 'am', 'an', 'indy', 'designer', 'from', 'MCR', 'about', 'to', 'launch', 'my', 'debut', 'clothing', 'line', 'for', 'diabetic', 'girls', 'would', 'you', 'mentor', 'me', 'X']

-----REPLACE EMOJIS AND EMOTICONS; REMOVE NON ASCII, TO LOWERCASE, REPLACE NUMBERS------:
['USER', 'hi', 'mary', 'i', 'am', 'an', 'indy', 'designer', 'from', 'mcr', 'about', 'to', 'launch', 'my', 'debut', 'clothing', 'line', 'for', 'diabetic', 'girls', 'would', 'you', 'mentor', 'me', 'x']

----------REMOVE STOPWORDS, LEMMATIZE AND STEM-------------:
['USER', 'hi', 'mari', 'indi', 'design', 'mcr', 'launch', 'debut', 'cloth', 'line', 'diabet', 'girl', 'would', 'mentor', 'x']



----------ORIGINAL----------:
Oman wellness: Prevent diabetic heart disease https://t.


-----REPLACED CONTRACTIONS, HASHTAGS, URLs AND USER MENTIONS, TOKENISE; REMOVE PUNCTUATIONS----:
['Missing', '14', 'year', 'old', 'last', 'seen', '9:30', 'am', 'this', 'date', 'At', 'his', 'home', '4600', 'North', 'Woodland', 'Hills', 'Dr', 'He', 'is', 'diabetic', 'and', 'had', 'URL']

INFO: Could not replace emoticon: :3 of the word: 9:30 (<class 'KeyError'>, KeyError('9:30',), <traceback object at 0x000000000A6DC488>)
-----REPLACE EMOJIS AND EMOTICONS; REMOVE NON ASCII, TO LOWERCASE, REPLACE NUMBERS------:
['missing', 'fourteen', 'year', 'old', 'last', 'seen', 'am', 'this', 'date', 'at', 'his', 'home', 'four thousand, six hundred', 'north', 'woodland', 'hills', 'dr', 'he', 'is', 'diabetic', 'and', 'had', 'URL']

----------REMOVE STOPWORDS, LEMMATIZE AND STEM-------------:
['miss', 'fourteen', 'year', 'old', 'last', 'see', 'date', 'home', 'four thousand, six hundr', 'north', 'woodland', 'hill', 'dr', 'diabet', 'URL']



----------ORIGINAL----------:
RT @UberFacts: If Americans stoppe

['people', 'who', 'make', 'the', 'common', 'diabetes', 'joke', 'on', 'a', 'food', 'item', 'that', 'contains', 'a', 'lot', 'of', 'sugar', 'do', 'not']

-----REPLACE EMOJIS AND EMOTICONS; REMOVE NON ASCII, TO LOWERCASE, REPLACE NUMBERS------:
['people', 'who', 'make', 'the', 'common', 'diabetes', 'joke', 'on', 'a', 'food', 'item', 'that', 'contains', 'a', 'lot', 'of', 'sugar', 'do', 'not']

----------REMOVE STOPWORDS, LEMMATIZE AND STEM-------------:
['peopl', 'make', 'common', 'diabet', 'joke', 'food', 'item', 'contain', 'lot', 'sugar']



----------ORIGINAL----------:
RT @prayerforce4: I pray against any affliction in your body including diabetes, cancer, heart disease, leukemia &amp; I declare you healed IJN…

-----REPLACED CONTRACTIONS, HASHTAGS, URLs AND USER MENTIONS, TOKENISE; REMOVE PUNCTUATIONS----:
['RT', 'USER', 'I', 'pray', 'against', 'any', 'affliction', 'in', 'your', 'body', 'including', 'diabetes', 'cancer', 'heart', 'disease', 'leukemia', 'I', 'declare', 'you', 'healed', 


-----REPLACED CONTRACTIONS, HASHTAGS, URLs AND USER MENTIONS, TOKENISE; REMOVE PUNCTUATIONS----:
['17.99', 'insulin', 'pump', 'bands', 'going', 'on', 'right', 'now', 'URL', 'URL']

-----REPLACE EMOJIS AND EMOTICONS; REMOVE NON ASCII, TO LOWERCASE, REPLACE NUMBERS------:
['17.99', 'insulin', 'pump', 'bands', 'going', 'on', 'right', 'now', 'URL', 'URL']

----------REMOVE STOPWORDS, LEMMATIZE AND STEM-------------:
['17.99', 'insulin', 'pump', 'band', 'go', 'right', 'URL', 'URL']



----------ORIGINAL----------:
Healthy Tip: 15 wonders of cucumber you didn't know

Good for diabetics. Patients of diabetes can enjoy cucumber... https://t.co/FFfedHLNCL

-----REPLACED CONTRACTIONS, HASHTAGS, URLs AND USER MENTIONS, TOKENISE; REMOVE PUNCTUATIONS----:
['Healthy', 'Tip', '15', 'wonders', 'of', 'cucumber', 'you', 'did', 'not', 'know', 'Good', 'for', 'diabetics', 'Patients', 'of', 'diabetes', 'can', 'enjoy', 'cucumber', 'URL']

-----REPLACE EMOJIS AND EMOTICONS; REMOVE NON ASCII, TO LOWERCASE, RE

### List of english stopwords that are deleted

In [49]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [49]:
# additional text cleaning

# Locate and correct common typos and misspellings
# Handling numbers and dates inside the text



In [50]:
prep.Constant_words

['URL',
 'USER',
 'EMOT_SMILE',
 'EMOT_LAUGH',
 'EMOT_LOVE',
 'EMOT_WINK',
 'EMOT_SAD_ANGRY',
 'EMOT_SURPRISED',
 'EMOT_CRY']